# comercio_ext_indices.tb_referencia_ncm_ppe
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_PPE_delta`
## 📌 Descrição do arquivo

Classificação **PPE** usada para categorizar produtos, com descrições em PT/EN.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_PPE`|Código PPE|Identificador PPE|
|`NO_PPE`|Nome (PT)|Descrição do PPE em português|
|`NO_PPE_MIN`|Nome (PT – minúsculo)|Variante textual|
|`NO_PPE_ING`|Nome (EN)|Descrição em inglês|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_PPE_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_ppe/"   
val silverTable =    "tb_referencia_ncm_ppe"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_PPE" , StringType ,  nullable = false ) ,
      StructField ( "NO_PPE" , StringType ,  nullable = false ) ,
      StructField ( "NO_PPE_MIN" , StringType ,  nullable = false ) ,
      StructField ( "NO_PPE_ING" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_PPE", trim(upper(col("CO_PPE").cast(StringType))))
  .withColumn("NO_PPE", col("NO_PPE").cast(StringType))
  .withColumn("NO_PPE_MIN", col("NO_PPE_MIN").cast(StringType))
  .withColumn("NO_PPE_ING", col("NO_PPE_ING").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/NCM_PPE_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_PPE").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_PPE"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_PPE"), col("NO_PPE"), col("NO_PPE_MIN"), col("NO_PPE_ING"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}